## Use case № 1
This notebook produces Effective Area (Aeff), Point Spread Function (PSF), Energy Dispersion (Edisp) files in .fits format from original KM3NeT simulation dst.root file

In [ ]:
from km3irf import build_irf
from astropy.io import fits

In [ ]:
from os import path

Define a path to your local `dst.root` file:

In [ ]:
folder = "/media/msmirnov/Data/working_space/data_files/files_cta_km3net"

In [ ]:
# data_path = "/run/media/msmirnov/iron_2tb/IRF_data_create/mcv5.1.km3_numuCC.ALL.dst.bdt.root"
# data_path = "/home/msmirnov/working_space/IRF_data_create/mcv5.1.km3_numuCC.ALL.dst.bdt.root"
# data_path = "/home/msmirnov/working_space/some_data/files_cta_km3net/mcv5.1.km3_numuCC.ALL.dst.bdt.root"
# data_path = "/run/media/msmirnov/DATA2/data_files/IRF_data_create/mcv5.1.km3_numuCC.ALL.dst.bdt.root"
data_path = path.join(folder, "mcv5.1.km3_numuCC.ALL.dst.bdt.root")

### Effective Area

Create BuildAeff object:

In [ ]:
test_irf = build_irf.DataContainer(data_path)

How many events in the file:

In [ ]:
test_irf.df.shape[0]

In [ ]:
test_irf.df.head(4)

Apply user defined cuts:

In [ ]:
test_irf.apply_cuts()
test_irf.df.shape[0]

Apply re-weighting procedure

In [ ]:
weighted_dict = test_irf.weight_calc(tag="nu")
weighted_dict.values()

Build .fits for effective area, it needs to specify the input pandas data frame.

In [ ]:
test_irf.build_aeff()

Check the output file with `gammapy`

In [ ]:
# install gammapy if it needs
#!pip install gammapy
from gammapy.irf import EffectiveAreaTable2D

In [ ]:
# Read effective area IRFs
aeff = EffectiveAreaTable2D.read("aeff.fits", hdu="EFFECTIVE AREA")
print(aeff)

In [ ]:
aeff.peek()

### Point Spread Function

In [ ]:
test_irf.build_psf()

Check the output file `psf.fits` with `gammapy`:

In [ ]:
from gammapy.irf import PSF3D

In [ ]:
psf = PSF3D.read("psf.fits", hdu="PSF_2D_TABLE")
print(psf)

In [ ]:
psf.peek()

### Energy dispertion

In [ ]:
test_irf.build_edisp(norm=True, smooth=False, smooth_norm=False)

Check the output file `edisp.fits` with `gammapy`:

In [ ]:
from gammapy.irf import EnergyDispersion2D

In [ ]:
edisp = EnergyDispersion2D.read("edisp.fits", hdu="EDISP_2D")
print(edisp)

In [ ]:
edisp.peek()

Write all created fits files to the one combined IRF file.

In [ ]:
from km3irf.utils import merge_fits

background is takken from `km3irf` data folder

In [ ]:
merge_fits(
    aeff_fits="aeff.fits",
    psf_fits="psf.fits",
    edisp_fits="edisp.fits",
    output_path=".",
    output_file="combined_IRF.fits",
)